In [ ]:
import json
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
def bootstrap(data: list[float], n: int = 10000):
    return np.random.choice(data, size=(n, len(data)), replace=True)

In [ ]:
res_path = Path('results/sam_vit_b_01ec64.pth.json')

with open(res_path, 'r') as f:
    res = json.load(f)

data = []
for r in res:
    data.append({
        'aAcc': np.mean(r['aAcc']),
        'mIoU': np.mean(r['mIoU']),
        'CLIP_MODEL_NAME': r['params']['CLIP_MODEL_PARAMS']['model'],
        'CLIP_MODEL_WEIGHTS': r['params']['CLIP_MODEL_PARAMS']['weights'],
        'CLIP_NORMALIZE_ATTENTION': r['params']['CLIP_NORMALIZE_ATTENTION'],
        'SAM_PROPOSER_NORM': r['params']['SAM_PROPOSER_NORM'],
        'SAM_PROPOSER_N_POINTS': r['params']['SAM_PROPOSER_N_POINTS'],
        'XAI_METHOD': r['params']['XAI_METHOD'],
    })

df = pd.DataFrame(data)

In [ ]:
for column in df.columns:
    if column in ['aAcc', 'mIoU', 'CLIP_MODEL_WEIGHTS']:
        continue

    results = []
    for value in df[column].unique():
        sub_df = df[df[column] == value]
        results.append({
            'column': column,
            'value': value,
            'mIoU_avg': sub_df['mIoU'].mean(),
            'mIoU_std': sub_df['mIoU'].std(),
            'aAcc_avg': sub_df['aAcc'].mean(),
            'aAcc_std': sub_df['aAcc'].std(),
        })

    results = pd.DataFrame(results)
    results = results.sort_values('mIoU_avg', ascending=False)
    print(results)
    print()

In [ ]:
df_sorted = df.sort_values('mIoU', ascending=False)
print(df_sorted)

In [ ]:
res_path = Path('results/sam_vit_l_0b3195.pth.json')

with open(res_path, 'r') as f:
    res = json.load(f)

data = []
for r in res:
    data.append({
        'aAcc': np.mean(r['aAcc']),
        'mIoU': np.mean(r['mIoU']),
        'CLIP_MODEL_NAME': r['params']['CLIP_MODEL_PARAMS']['model'],
        'CLIP_MODEL_WEIGHTS': r['params']['CLIP_MODEL_PARAMS']['weights'],
        'CLIP_NORMALIZE_ATTENTION': r['params']['CLIP_NORMALIZE_ATTENTION'],
        'SAM_PROPOSER_NORM': r['params']['SAM_PROPOSER_NORM'],
        'SAM_PROPOSER_N_POINTS': r['params']['SAM_PROPOSER_N_POINTS'],
        'XAI_METHOD': r['params']['XAI_METHOD'],
    })

df = pd.DataFrame(data)

In [ ]:
for column in df.columns:
    if column in ['aAcc', 'mIoU', 'CLIP_MODEL_WEIGHTS']:
        continue

    results = []
    for value in df[column].unique():
        sub_df = df[df[column] == value]
        results.append({
            'column': column,
            'value': value,
            'mIoU_avg': sub_df['mIoU'].mean(),
            'mIoU_std': sub_df['mIoU'].std(),
            'aAcc_avg': sub_df['aAcc'].mean(),
            'aAcc_std': sub_df['aAcc'].std(),
        })

    results = pd.DataFrame(results)
    results = results.sort_values('mIoU_avg', ascending=False)
    print(results)
    print()

In [ ]:
df_sorted = df.sort_values('mIoU', ascending=False)
print(df_sorted)

In [ ]:
# res_path = Path('results/sam_vit_l_0b3195.pth.json')
# res_path = Path('results/sam_vit_b_01ec64.pth.json')
res_path = Path('results/sam_vit_h_4b8939.pth.json')

with open(res_path, 'r') as f:
    res = json.load(f)

data = []
for r in res:
    data.append({
        'aAcc': np.mean(r['aAcc']),
        'mIoU': np.mean(r['mIoU']),
        'CLIP_MODEL_NAME': r['params']['CLIP_MODEL_PARAMS']['model'],
        'CLIP_MODEL_WEIGHTS': r['params']['CLIP_MODEL_PARAMS']['weights'],
        'CLIP_NORMALIZE_ATTENTION': r['params']['CLIP_NORMALIZE_ATTENTION'],
        'SAM_PROPOSER_NORM': r['params']['SAM_PROPOSER_NORM'],
        'SAM_PROPOSER_N_POINTS': r['params']['SAM_PROPOSER_N_POINTS'],
        'XAI_METHOD': r['params']['XAI_METHOD'],
    })

df = pd.DataFrame(data)

col = 'SAM_PROPOSER_N_POINTS'
df_col = df[['aAcc', 'mIoU', col]]

n_bootstraps = 10_000
n_labels = len(df_col[col].unique())
max_val = df_col['mIoU'].max()
fig, axes = plt.subplots(n_labels, 1, figsize=(10, 10))
for i, value in enumerate(df_col[col].unique()):
    ax = axes[i]
    df_selected = df_col[df_col[col] == value]
    mious = df_selected['mIoU'].to_list()
    bootstraped = np.mean(bootstrap(mious, n_bootstraps), axis=1)
    sns.histplot(bootstraped, kde=True, label=value, ax=axes[i])
    mean_bootstrapped = np.mean(bootstraped)
    ax.axvline(mean_bootstrapped, color='red', linestyle='--')
    ax.set_title(f'{col}={value}')
    ax.set_xlabel('mIoU')
    ax.set_ylabel('Frequency')
    ax.legend()
    ax.set_xlim(0, max_val)
plt.tight_layout()